# Main code

In [18]:
from classes import Species, Observation
sc.addPyFile("classes.py")


        
        
if __name__ == "__main__":
    global species
    data = sc.textFile("/home/WUR/steen176/Downloads/test.dat")
    #data = sc.textFile('hdfs://scomp1334:9000/user/steen176/csv/butterflies.csv')

    corrected = data.map(lambda line: correctLine(line))
    species = extractSpecies(corrected)
    
    
    #rdd.map did not work here
    map(getObservation, corrected.toLocalIterator())
    
    to_cluster = sc.parallelize(species.keys())
    
    
    #test for 1 organism
    cl =  cluster(to_cluster.take(1)[0])

    parallel_centers = sc.parallelize(cl.clusterCenters)
    print(parallel_centers.take(1))
        
    

[u'Falco subbuteo', u'Cuculus canorus']
[array([ 90.076225  ,   1.36562839])]


# Utility

In [4]:
def getSpecies(line):
    line = line.split(',')
    return line[0]

def correctLine(line):
    line = line.split(',')
    if len(line) != 10:
        return ','.join(line[0:4] + [" ".join(line[4:6])]+ line[6:])
    else:
        return ','.join(line)

def getObservation(line):
    line = line.split(',')
    specie = line[0]
    for i in xrange(int(line[5])):
        longti = line[8]
        lati = line[9]
        obs = Observation(longti,lati)
        species[specie].addObservation(obs)
    
    
def createSpecieObjects(species_list):
    species_objects = {}
    for specie in species_list:
        obj = Species(specie)
        species_objects[specie] = obj
    return species_objects


def extractSpecies(rdd):
    species = rdd.map(getSpecies)
    species_list  = list(set([x for x in species.toLocalIterator()]))
    return createSpecieObjects(species_list)
"""
Curculus canorus: species
137: species code
1: code species group
Ter plaatse: activity
Koekoeken: family
1: how many
2014-05-16: date of observation
21:34: time of observation
6.07659218423424,0.0523654652736141: location
"""

'\nCurculus canorus: species\n137: species code\n1: code species group\nTer plaatse: activity\nKoekoeken: family\n1: how many\n2014-05-16: date of observation\n21:34: time of observation\n6.07659218423424,0.0523654652736141: location\n'

# Clustering

In [2]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.mllib.linalg import SparseVector
from math import sqrt
NUM_CLUSTER = 4

def kmeans(sparse_vectors):
    #sparsevectors should be a rdd containing sparsevectors
    return KMeans.train(sparse_vectors, NUM_CLUSTER, maxIterations=20, runs=100, initializationMode="random")

def cluster(specie):
    specie = species[specie]
    vectors = specie.getVectorRDD()
    return kmeans(sc.parallelize(vectors))


    
def error(point):
    center = clusters.centers[clusters.predict(point)]
    denseCenter = DenseVector(numpy.ndarray.tolist(center))
    return sqrt(sum([x**2 for x in (DenseVector(point.toArray()) - denseCenter)]))

    WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
    print(WSSSE)

# Comparing clusters

In [ ]:
def centerdiffs(centerA, centerB, DIFF=0.1):
    longA = centerA[0]
    longB = centerB[0]
    lattA = centerA[1]
    lattB = centerB[1]
    
    longDiff = numpy.subtract(longA, longB)
    lattDiff = numpy.subtract(lattA, lattB)
    
    if longDiff < DIFF and lattDiff < DIFF:
        return True
    else:
        return False